In [1]:
import random

import matplotlib.pyplot as plt

import networkx as nx

from deap_er import base, creator, tools

from py_wake.examples.data.iea37 import IEA37_WindTurbines, IEA37Site
from py_wake.literature import Jensen_1983

from evolve import evolve

In [2]:
site = IEA37Site()
wind_directions = range(0, 360, 10)
wind_probabilities = [1 / len(wind_directions)] * len(wind_directions)

site.default_wd = wind_directions
site.default_wd_probabilities = wind_probabilities  

pairs = IEA37_WindTurbines()

model = Jensen_1983(site, pairs)

In [3]:
creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti )

In [4]:
toolbox = base.Toolbox()

boundary_length = site.boundary_radius / (2 ** 0.5)
toolbox.register("attr_coordinate", random.uniform, -boundary_length, boundary_length)

MIN_TURBINES = 2
MAX_TURBINES = 9
def individual():
    num_attrs = random.randint(MIN_TURBINES, MAX_TURBINES) * 2
    return creator.Individual(
        [toolbox.attr_coordinate() for _ in range(num_attrs)]
    )
toolbox.register("individual", individual)

toolbox.register("population", tools.init_repeat, list, toolbox.individual)

In [5]:
COST_PER_TURBINE = 2_000_000
COST_PER_METER_WIRING = 50

def euclidean_distance(x1, y1, x2, y2):
    return ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

def evaluate(ind):
    x, y = ind[::2], ind[1::2]
    result = model(x, y)
    aep_gwh = result.aep().sum().values

    total_turbine_cost = COST_PER_TURBINE * len(ind) / 2
    
    graph = nx.Graph()

    pairs = list(zip(x, y))
    for a, p in enumerate(pairs):
        graph.add_node(a, pos=p)

    for a in range(len(pairs)):
        for b in range(a + 1, len(pairs)):
            x1, y1 = pairs[a]
            x2, y2 = pairs[b]
            dist = euclidean_distance(x1, y1, x2, y2)
            graph.add_edge(a, b, dist=dist)

    mst = nx.minimum_spanning_tree(graph, weight='dist')
    total_wiring_cost = mst.size(weight="dist") * COST_PER_METER_WIRING

    return aep_gwh * 1_000_000, total_turbine_cost + total_wiring_cost

toolbox.register("evaluate", evaluate)

In [6]:
def clamp(_min, _max):
    def wrapper(func):
        def wrapped(*args, **kwargs):
            inds: tuple = func(*args, **kwargs)
            for ind in inds:
                for i in range(len(ind)):
                    if ind[i] > _max:
                        ind[i] = _max
                    elif ind[i] < _min:
                        ind[i] = _min
            return inds
        return wrapped
    return wrapper

In [ ]:

def mutate(ind):
    if random.random() < 0.8:
        ind, = tools.mut_gaussian(ind, 0, 0.5 * site.boundary_radius, 2.0 / len(ind))
    else:
        if (random.random() < 0.5 and len(ind) < MAX_TURBINES * 2) or len(ind) == MIN_TURBINES * 2:
            for _ in range(2):
                ind.append(toolbox.attr_coordinate())
        else:
            del ind[-2:]
    return ind,

toolbox.register("mutate", mutate)
toolbox.decorate("mutate", clamp(-boundary_length, boundary_length))


In [8]:
toolbox.register("mate", tools.cx_two_point)
toolbox.decorate("mate", clamp(-boundary_length, boundary_length))

toolbox.register("select", tools.sel_tournament, contestants=4)

In [9]:
NUM_GEN = 100
POP_SIZE = 100
PC = 0.0
PM = 0.7

pop = toolbox.population(POP_SIZE)
pop, logbook = evolve(toolbox, pop, pc=PC, pm=PM, num_elitism=round(POP_SIZE * 0.1), num_gen=NUM_GEN)    

gen	max        	pareto_front                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

ValueError: Turbines 6 and 7 are at the same position
Turbines 7 and 6 are at the same position

In [ ]:
pareto_front = logbook.select("pareto_front")[-1]

for ind in pareto_front:
    x, y = ind[::2], ind[1::2]
    plt.xlim(-boundary_length, boundary_length)
    plt.ylim(-boundary_length, boundary_length)
    plt.scatter(x, y)
    plt.show()

In [ ]:
pfs = logbook.select("pareto_front")

for pf in pfs:
    x=[]
    y=[]
    for ind in pf:
        x.append(ind.fitness.values[0])
        y.append(ind.fitness.values[1])
    plt.ylim(0, 3 * 10**7)
    plt.xlim(0, 3 * 10**8) 
    plt.scatter(x, y)
    plt.show()
    